In [1]:
import random
from mido import Message, MidiFile, MidiTrack, MetaMessage
import stilus.midi.utils as utl
import stilus.models as m
import stilus.midi.imagine as img
import numpy as np

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from stilus.data.sets import MidiDataset
from torch.utils.data import DataLoader

In [3]:
#import a midi file
file_path = "midi/training/mozart/mz_545_3.mid"
tokens = file_path.split("/")
file_name = tokens[len(tokens)-1]
file_name_no_ext = file_name.split(".")[0]

mid = MidiFile(file_path)
file_name_no_ext

'mz_545_3'

In [4]:
# print all events
for i, track in enumerate(mid.tracks):
    print('Track {}: {}'.format(i, track.name))
    for msg in track:
        #if msg.type == "note_on":
        print(msg)

Track 0: Klaviersonate KV 545 3. Satz
<meta message track_name name='Klaviersonate KV 545 3. Satz' time=0>
<meta message copyright text='Copyright © 2006 by Bernd Krueger' time=0>
<meta message text text='Mozart' time=0>
<meta message text text='Veröffentlicht: 1.10.2006\n' time=0>
<meta message text text='Update: 29.9.2013\n' time=0>
<meta message text text='Dauer: 1:44 Minuten\n' time=0>
<meta message time_signature numerator=2 denominator=4 clocks_per_click=24 notated_32nd_notes_per_beat=8 time=0>
<meta message key_signature key='C' time=0>
<meta message set_tempo tempo=589623 time=0>
<meta message set_tempo tempo=647878 time=480>
<meta message set_tempo tempo=623895 time=480>
<meta message set_tempo tempo=651466 time=240>
<meta message set_tempo tempo=620604 time=240>
<meta message set_tempo tempo=630583 time=240>
<meta message set_tempo tempo=620604 time=240>
<meta message set_tempo tempo=644399 time=240>
<meta message set_tempo tempo=617347 time=240>
<meta message set_tempo tempo

note_on channel=0 note=60 velocity=0 time=120
note_on channel=0 note=64 velocity=51 time=0
note_on channel=0 note=64 velocity=0 time=120
note_on channel=0 note=55 velocity=57 time=0
note_on channel=0 note=55 velocity=0 time=120
note_on channel=0 note=65 velocity=51 time=0
note_on channel=0 note=65 velocity=0 time=120
note_on channel=0 note=62 velocity=55 time=0
note_on channel=0 note=62 velocity=0 time=120
note_on channel=0 note=65 velocity=58 time=0
note_on channel=0 note=65 velocity=0 time=120
note_on channel=0 note=60 velocity=67 time=0
note_on channel=0 note=60 velocity=0 time=120
note_on channel=0 note=64 velocity=58 time=0
note_on channel=0 note=64 velocity=0 time=120
note_on channel=0 note=55 velocity=60 time=0
note_on channel=0 note=55 velocity=0 time=120
note_on channel=0 note=60 velocity=56 time=0
note_on channel=0 note=60 velocity=0 time=120
note_on channel=0 note=52 velocity=62 time=0
note_on channel=0 note=52 velocity=0 time=120
note_on channel=0 note=55 velocity=55 time=0

In [5]:
timeseries_tensor = utl.convert_midi_to_time_series(mid,5,5,8)
print(timeseries_tensor.shape)
n = 136
print(timeseries_tensor[:,n-32:n])

(5, 2608)
[[81.  0. 79.  0. 77.  0. 76.  0. 73.  0. 74.  0. 76.  0. 77.  0. 69.  0.
  72.  0. 71.  0. 74.  0. 72.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]]


In [6]:
version ="version_5"
epochs = "5"
net = m.ConvNet_1_0_2().load_from_checkpoint("tb_logs/ConvNet_1_0_2_mozart/"+ version +"/checkpoints/epoch="+ epochs +".ckpt")
#net = m.TransformerNet_1_0_2().load_from_checkpoint("tb_logs/TransformerNet_1_0_2_mozart_test/"+ version +"/checkpoints/epoch="+ epochs +".ckpt")
net.eval( )

ConvNet_1_0_2(
  (criterion): MSELoss()
  (conv1): Conv1d(5, 10, kernel_size=(4,), stride=(1,))
  (fc0): Linear(in_features=610, out_features=64, bias=True)
  (conv2): Conv1d(1, 10, kernel_size=(4,), stride=(1,))
  (fc1): Linear(in_features=300, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=5, bias=True)
)

In [7]:
#set path to dataloaders
net.set_data_path("data/mozart_test")

In [11]:
timeseries_tensor.shape

(5, 2608)

In [14]:
generated_series = img.imagine_midi(timeseries_tensor, net, ["128:160", "200:280", "400:448", "1200:1280", "1600:1760","2000:2160"])
print(generated_series[:,136:172])
print(generated_series.shape)

2608
[[82  0 69  0 85  0 67  0 76  0 55  0 55  0 48  1 73  0 59  0 71  0 55  0
  71  0  0  0  0  0  0  0 72  0 74  0]
 [ 5  0  0  0  7  0  1  0  9  1  0  0  2  0  0  0  7  0  2  0  6  0  3  0
  74  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  1  0  0  0  1  0  0  0  1  0  1  0  1  0  1  0  1  0  1  0
   0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  1  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0]]
(5, 2608)


In [15]:
out_path = 'midi/weight_analysis/' + file_name_no_ext +"_" + net.name + "_" + version + '.mid'
utl.write_midi_from_series(generated_series, 2, out_path )

Creating midi file:  midi/weight_analysis/mz_545_3_ConvNet_1_0_2_mozart_test_version_5.mid  from series
